In [1]:
import numpy as np
import matplotlib.pyplot as plt
outdir = r"C:\Users\nickd\PyFluid\gifs"

In [ ]:
#1 Dimension MHD, Compressible, Adiabatic, Inviscid, No Viscosity, No Resistivity
class fluid1DMHD:
   def __init__(self,x0,x1,nx,ici,bci,Nsteps):
      #Initialize the base grid, ghost cells and interior.
      self.Nsteps = int(Nsteps)
      self.x0,self.x1 = x0,x1
      self.nx = int(nx)
      self.ici = ici; self.bci = bci;
      self.ng = 2 #number of ghost cells
      self.xg = np.linspace(x0,x1,nx) #grid with ghost cells
      self.dx = self.xg[1]-self.xg[0];
      self.xi = self.xg[self.ng:-self.ng] #interior grid
      self.nxi = self.nx - 2*self.ng #number of interior grid points
      self.nxifs = self.nxi + 1 #number of interfaces
      self.L = self.x1-self.x0 #domain length
      self.Li = self.xg[-self.ng] - self.xg[self.ng] #interior domain length
      self.gmask = np.zeros(nx,dtype=bool);
      self.gmask[:self.ng] = True; self.gmask[-self.ng:] = True
      self.imask = ~self.gmask
      self.lhgsl = slice(0,self.ng) #left side ghost cell slice
      self.lhisl = slice(self.ng,2*self.ng) #left side interior slice
      self.rhgsl = slice(-self.ng,None) #right side ghost cell slice
      self.rhisl = slice(-2*self.ng,-self.ng) #right side interior
      #1D MHD we have (rho, rhovx, rhovy, rhovz, E, By, Bz) as our conserved variables
      self.nvar = 7;
      self.Uvec = np.zeros((self.nvar,nx)) #conserved variable state vector
      self.Fvec = np.zeros((self.nvar,nx)) #flux vector

      #Physics Assumptions
      self.gamma = 5/3 #adiabatic index
      self.kappa = 1.0 #entropy
      self.rhob = 1.0 #background density
      self.pres0 = self.kappa*self.rhob**self.gamma #background pressure
      self.cs20 = self.gamma*self.kappa*self.rhob**(self.gamma-1) #background sound speed squared

      self.apply_ics()
      self.apply_bcs()
   def apply_ics(self):
      self[0,:] += self.rhob #background density
      self
      

   def apply_bcs(self):
      #LHS Periodic
      if self.bci[0] == 0:
         self.Uvec[:,self.lhgsl] = self.Uvec[:,self.rhisl]
      #RHS Periodic
      if self.bci[1] == 0:
         self.Uvec[:,self.rhgsl] = self.Uvec[:,self.lhisl]
     
      #LHS Outflow
      if self.bci[0] == 1:
         self.Uvec[:,self.lhgsl] = self.Uvec[:,self.lhisl]
      #RHS Outflow
      if self.bci[1] == 1:
         self.Uvec[:,self.rhgsl] = self.Uvec[:,self.rhisl]
      
      #LHS Reflective
      if self.bci[0] == 2:
         self.Uvec[:,self.lhgsl] = self.Uvec[:,self.lhisl]
         self.Uvec[1:4,self.lhgsl] *= -1
      #RHS Reflective
      if self.bci[1] == 2:
         self.Uvec[:,self.rhgsl] = self.Uvec[:,self.rhisl]
         self.Uvec[1:4,self.rhgsl] *= -1
      if self.bci[0] not in [0,1,2] or self.bci[1] not in [0,1,2]:
         raise ValueError("Boundary condition not recognized. Use 0 = periodic, 1 = outflow, 2 = reflective")


x0 = -1.0; x1 = 1.0; nx= 400;
#Initial conditions, 0 = sod, 1 = blast wave, 2 = alfven wave
ici = 0;
#Boundary conditions, 0 = periodic, 1 = outflow, 2 = reflective
bci = [0,0]; 

Nsteps = 1e2;
state = fluid1DMHD(x0,x1,nx,ici,bci,Nsteps);
